In [ ]:
!pip install transformers

     |████████████████████████████████| 1.0MB 5.3MB/s 
     |████████████████████████████████| 3.0MB 30.7MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3ea24b465b86391b8d15a5806d0334c1798ecf23124c30b8766227299509d86f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install datasets

     |████████████████████████████████| 1.8MB 4.5MB/s 
     |████████████████████████████████| 245kB 31.7MB/s 
     |████████████████████████████████| 17.3MB 202kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
import tensorflow as tf
from transformers import *
from transformers import TFBertForSequenceClassification, BertTokenizer, glue_convert_examples_to_features
import numpy as np
import json

In [ ]:
from transformers.data.processors.utils import InputExample

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_polarity")

def createDataset(df):
  datalist = [InputExample(guid=idx, text_a = data['text'], label = data['label']) for idx, data in enumerate(df)]
  return datalist

train_data = createDataset(dataset['train'])
test_data = createDataset(dataset['test'])

num_train_examples = len(train_data)
num_dev_examples = len(test_data)
num_labels = 2

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/2b33212d89209ed1ea0522001bccc5f5a5c920dd9c326f3c828e67a22c51a98c. Subsequent calls will reuse this data.


In [ ]:
def getDataset(df):
  all_input_ids = np.array([f.input_ids for f in df])
  all_attention_mask = np.array([f.attention_mask for f in df])
  all_token_type_ids = np.array([f.token_type_ids for f in df])
  all_labels = np.array([f.label for f in df])
  trainX = tf.data.Dataset.from_tensor_slices({'input_ids' : all_input_ids, 'attention_mask' : all_attention_mask, 'token_type_ids': all_token_type_ids})
  trainY = tf.data.Dataset.from_tensor_slices(all_labels)
  return tf.data.Dataset.zip((trainX, trainY))

In [ ]:
def prepare_and_train(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  max_seq_length = 96
  train_batch_size = 8 
  eval_batch_size = 8 

  train = glue_convert_examples_to_features(examples=train_data[:10000], tokenizer = tokenizer, max_length=max_seq_length, task='sst-2', label_list=[0,1])
  test = glue_convert_examples_to_features(examples=test_data[:1000], tokenizer = tokenizer, max_length=max_seq_length, task='sst-2', label_list=[0,1])

  btrain = getDataset(train)
  btrain = btrain.shuffle(100).batch(train_batch_size)

  btest = getDataset(test)
  btest = btest.shuffle(100).batch(eval_batch_size)

  learning_rate = 2e-5 

  #Typically these type of models are finetuned for 3 epochs. This can be increased for small datasets and decreased for large datasets.
  num_epochs = 3  

  # Initialise a Model for Sequence Classification with 2 labels
  config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
  model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

  # Optimizer and loss
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  # Metrics and callbacks
  metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]

  # Compute some variables
  train_steps_per_epoch = int(num_train_examples / train_batch_size)
  dev_steps_per_epoch = int(num_dev_examples / eval_batch_size)


  # Compile model
  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  # Train the model
  history = model.fit(btrain, epochs=num_epochs, validation_data=btest)

  # Print some information about the training
  print(f'\nThe training has finished training after {num_epochs} epochs.')
  print('\nThe history contains the accuracy and loss at every epoch:')
  print(json.dumps(history.history, indent=4))

In [ ]:
prepare_and_train('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1250/1250 [==============================] - 243s 194ms/step - loss: 0.2893 - accuracy: 0.8773 - val_loss: 0.2437 - val_accuracy: 0.8930
Epoch 2/3
1250/1250 [==============================] - 242s 193ms/step - loss: 0.1387 - accuracy: 0.9511 - val_loss: 0.2740 - val_accuracy: 0.9040
Epoch 3/3
1250/1250 [==============================] - 242s 193ms/step - loss: 0.0693 - accuracy: 0.9758 - val_loss: 0.5198 - val_accuracy: 0.8760

The training has finished training after 3 epochs.

The history contains the accuracy and loss at every epoch:
{
    "loss": [
        0.28933241963386536,
        0.13871026039123535,
        0.06934858113527298
    ],
    "accuracy": [
        0.8773000240325928,
        0.9510999917984009,
        0.9757999777793884
    ],
    "val_loss": [
        0.24368835985660553,
        0.2740303874015808,
        0.5197854042053223
    ],
    "val_accuracy": [
        0.8930000066757202,
        0.9039999842643738,
        0.8759999871253967
    ]
}


In [ ]:
prepare_and_train('xlnet-base-cased')

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:211: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1250/1250 [==============================] - 292s 234ms/step - loss: 0.2689 - accuracy: 0.8923 - val_loss: 0.2056 - val_accuracy: 0.9200
Epoch 2/3
1250/1250 [==============================] - 290s 232ms/step - loss: 0.1632 - accuracy: 0.9366 - val_loss: 0.2195 - val_accuracy: 0.9170
Epoch 3/3
1250/1250 [==============================] - 290s 232ms/step - loss: 0.0957 - accuracy: 0.9656 - val_loss: 0.2940 - val_accuracy: 0.9080

The training has finished training after 3 epochs.

The history contains the accuracy and loss at every epoch:
{
    "loss": [
        0.268883615732193,
        0.1632051020860672,
        0.09574691206216812
    ],
    "accuracy": [
        0.892300009727478,
        0.9366000294685364,
        0.9656000137329102
    ],
    "val_loss": [
        0.20556089282035828,
        0.21946974098682404,
        0.29403626918792725
    ],
    "val_accuracy": [
        0.9200000166893005,
        0.9169999957084656,
        0.9079999923706055
    ]
}
